# GPT와 미국 주식 이야기하기

- 요새 마이크로소프트 주식 얼마야?
- 테슬라 주식을 지금 더 사야할까 팔아야할까?

등 위와 같은 대화를 하기 위해서는 최신 주식 정보를 GPT가 알고 있어야 함

`yfinance` 라이브러리를 통해 미국 주식 정보를 가져와서 GPT와 대화할 때 사용하는 실습 진행

## yfinance 라이브러리 설치
- 실제 자신의 개발환경에서 실습하고 싶은 사람은 가상 환경 실행 후 직접 설치

In [1]:
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.


- 함수 출력에 사용될 라이브러리 추가 설치

In [2]:
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.


## 라이브러리 불러오기 및 예제

- Ticker란?

  - `yfinance` 라이브러리에서 특정 주식의 고유 식별자인 티커 심볼을 나타내는 객체
  - Ticker 객체를 사용해서 특정 주식의 데이터를 쉽게 가져와 주가, 거래량 등의 금융 정보를 조회하고 분석할 수 있음

In [3]:
import yfinance as yf

# 테슬라 (TSLA)에 대한 Ticker 객체 생성
tsla = yf.Ticker("TSLA")

# Ticker 객체에 대한 정보 출력 (.py에서 실행할 때는 print(tsla.info)로 사용)
tsla.info

{'address1': '1 Tesla Road',
 'city': 'Austin',
 'state': 'TX',
 'zip': '78725',
 'country': 'United States',
 'phone': '512 516 8177',
 'website': 'https://www.tesla.com',
 'industry': 'Auto Manufacturers',
 'industryKey': 'auto-manufacturers',
 'industryDisp': 'Auto Manufacturers',
 'sector': 'Consumer Cyclical',
 'sectorKey': 'consumer-cyclical',
 'sectorDisp': 'Consumer Cyclical',
 'longBusinessSummary': 'Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. The company operates in two segments, Automotive; and Energy Generation and Storage. The Automotive segment offers electric vehicles, as well as sells automotive regulatory credits; and non-warranty after-sales vehicle, used vehicles, body shop and parts, supercharging, retail merchandise, and vehicle insurance services. This segment also provides sedans and sport utility vehicles through direct and used vehic

In [4]:
hist = tsla.history(period="5d") # 5일간의 주가 데이터를 가져옴
display(hist) # 데이터 출력

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-10-27 00:00:00-04:00,439.980011,460.160004,438.690002,452.420013,105867500,0.0,0.0
2025-10-28 00:00:00-04:00,454.779999,467.000000,451.600006,460.549988,80185700,0.0,0.0
2025-10-29 00:00:00-04:00,462.500000,465.700012,452.649994,461.510010,67983500,0.0,0.0
2025-10-30 00:00:00-04:00,451.049988,455.059998,439.609985,440.100006,72447900,0.0,0.0
2025-10-31 00:00:00-04:00,446.750000,458.000000,443.690002,456.559998,82980800,0.0,0.0


In [5]:
tsla.recommendations # 추천 정보 출력

,period,strongBuy,buy,hold,sell,strongSell
0,0m,5,15,17,7,3
1,-1m,5,15,17,6,2
2,-2m,5,14,18,7,3


## GPT에서 사용할 함수 정의
- `get_current_time` : 타임존 기반 현재 시간 반환
- `get_yf_stock_info` : 종목 정보 반환
- `get_yf_stock_history` : 주가 정보 반환
- `get_yf_stock_recommendations` : 추천 정보 반환

### 함수 정의에 필요한 라이브러리 불러오기

In [6]:
from datetime import datetime
import pytz
import yfinance as yf
import tabulate

- 타임존 기반 현재 시간을 반환하는 함수 정의

In [7]:
def get_current_time(timezone: str = 'Asia/Seoul'):
    tz = pytz.timezone(timezone) # 타임존 설정
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    now_timezone = f'{now} {timezone}'
    print(now_timezone)
    return now_timezone

- 주식 정보 관련 함수 정의

In [9]:
def get_yf_stock_info(ticker: str):
    stock = yf.Ticker(ticker)
    info = stock.info
    print(info)
    return str(info)

def get_yf_stock_history(ticker: str, period: str):
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    history_md = history.to_markdown() # 데이터프레임을 마크다운 형식으로 변환
    print(history_md)
    return history_md

def get_yf_stock_recommendations(ticker: str):
    stock = yf.Ticker(ticker)
    recommendations = stock.recommendations
    recommendations_md = recommendations.to_markdown() # 데이터프레임을 마크다운 형식으로 변환
    print(recommendations_md)
    return recommendations_md

##  GPT를 위해 사용할 함수 설명 추가하기

- GPT API 호출 시 펑션 콜링할 도구를 정의해야 함

- 콜링할 함수의 메타데이터를 JSON 형식으로 정의하며, 도구의 이름, 설명, 매개변수 등을 포함하고 있음

In [10]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "해당 타임존의 날짜와 시간을 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'timezone': {
                        'type': 'string',
                        'description': '현재 날짜와 시간을 반환할 타임존을 입력하세요. (예: Asia/Seoul)',
                    },
                },
                "required": ['timezone'],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_yf_stock_info",
            "description": "해당 종목의 Yahoo Finance 정보를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'ticker': {
                        'type': 'string',
                        'description': 'Yahoo Finance 정보를 반환할 종목의 티커를 입력하세요. (예: AAPL)',
                    },
                },
                "required": ['ticker'],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_yf_stock_history",
            "description": "해당 종목의 Yahoo Finance 주가 정보를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'ticker': {
                        'type': 'string',
                        'description': 'Yahoo Finance 주가 정보를 반환할 종목의 티커를 입력하세요. (예: AAPL)',
                    },
                    'period': {
                        'type': 'string',
                        'description': '주가 정보를 조회할 기간을 입력하세요. (예: 1d, 5d, 1mo, 1y, 5y)',
                    },
                },
                "required": ['ticker', 'period'],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_yf_stock_recommendations",
            "description": "해당 종목의 Yahoo Finance 추천 정보를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'ticker': {
                        'type': 'string',
                        'description': 'Yahoo Finance 추천 정보를 반환할 종목의 티커를 입력하세요. (예: AAPL)',
                    },
                },
                "required": ['ticker'],
            },
        }
    },
]

## 함수 실행해보기

아래 코드를 실행해서, 새로 추가한 함수들 기능 확인하기

In [11]:
get_yf_stock_history('AAPL', '5d') # 애플 주가 정보
print('----')
get_yf_stock_recommendations('AAPL')

| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |
|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|
| 2025-10-27 00:00:00-04:00 | 264.88 | 269.12 | 264.65 |  268.81 | 4.48882e+07 |           0 |              0 |
| 2025-10-28 00:00:00-04:00 | 268.99 | 269.89 | 268.15 |  269    | 4.15348e+07 |           0 |              0 |
| 2025-10-29 00:00:00-04:00 | 269.28 | 271.41 | 267.11 |  269.7  | 5.10867e+07 |           0 |              0 |
| 2025-10-30 00:00:00-04:00 | 271.99 | 274.14 | 268.48 |  271.4  | 6.98865e+07 |           0 |              0 |
| 2025-10-31 00:00:00-04:00 | 276.99 | 277.32 | 269.16 |  270.37 | 8.60967e+07 |           0 |              0 |
----
|    | period   |   strongBuy |   buy |   hold |   sell |   strongSell |
|---:|:---------|------------:|------:|-------:|-------:|-------------:|
|  0 | 0m       |           5 |    24 |     15 |      1 |        

'|    | period   |   strongBuy |   buy |   hold |   sell |   strongSell |\n|---:|:---------|------------:|------:|-------:|-------:|-------------:|\n|  0 | 0m       |           5 |    24 |     15 |      1 |            3 |\n|  1 | -1m      |           5 |    23 |     15 |      1 |            3 |\n|  2 | -2m      |           5 |    22 |     15 |      1 |            1 |'

## GPT 실행

### GPT 실행을 위해 필요한 라이브러리 불러오기

In [12]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
# import streamlit as st

### 환경 변수에서 API 키 가져오기

In [13]:
load_dotenv()
api_key = os.getenv("OPEN_API_KEY")
client = OpenAI(api_key=api_key)  # OpenAI 클라이언트 인스턴스 생성

### GPT에 tools 정보 포함하기
- 기존 `get_ai_response` 함수 파라미터에 tools 추가
- tools = tools : 위에서 `tools`로 정의한 함수들을 사용할 수 있음

In [14]:
def get_ai_response(messages, tools=None):
    response = client.chat.completions.create(
        model="gpt-4o",  # 사용할 모델 지정
        messages=messages,  # 대화 기록 전달
        tools=tools,  # 사용 가능한 도구 전달
    )
    return response  # 생성된 응답 반환

### get_ai_response 함수 실행 준비하기
아래 코드는 사용자 입력을 받아 GPT와 대화를 진행하는 내용

- `input`: 사용자로부터 입력을 받음
- `exit`: 사용자가 "exit"을 입력하면 대화를 종료
- `messages`: 대화 맥락을 유지할 수 있도록 계속 append 시켜줌, 사용자와 GPT의 메시지를 저장함

GPT가 도구를 호출해야 할 때 `tool_calls`가 포함됨

- `tool_calls`?
  - GPT가 특정 작업을 수행하기 위해 도구를 호출해야 한다고 판단하면 응답에 `tool_calls`가 포함됨

  - ex) 사용자가 "현재 시간을 알려줘"라고 요청하면 GPT는 `get_current_time` 함수를 호출해야 한다고 판단할 수 있음

- `tool_calls` 처리 과정
  - `tool_calls`가 있는지 확인

  - 호출해야 할 함수 이름과 매개변수 추출

  - 해당 함수를 실행하고 결과를 대화 기록에 추가

- 다음 셀에서 실행한 질문들
  - 1) 안녕하세요
  - 2) 현재 서울 시간이 몇시인가요?
  - 3) 현재 애플의 주가 정보를 알려주세요
  - 4) 테슬라를 사야할까요 팔아야할까요?
  - 5) 마이크로소프트 1주에 얼마죠?
  - 6) exit

이외에도 질문들을 바꿔보며 tool_calls가 언제 포함되는지 비교해보고, 다양한 주식 관련 에이전트 답변을 확인해보기

In [15]:
# 초기 메시지 설정
messages = [
    {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},  # 시스템 메시지
]

print("GPT와 대화를 시작합니다. 'exit'을 입력하면 종료됩니다.")

while True:
    # 사용자 입력 받기
    user_input = input("사용자: ")
    if user_input.lower() == "exit":  # 'exit' 입력 시 종료
        print("대화를 종료합니다.")
        break

    # 사용자 메시지를 대화 기록에 추가
    messages.append({"role": "user", "content": user_input})

    # GPT 응답 생성
    ai_response = get_ai_response(messages, tools=tools)
    ai_message = ai_response.choices[0].message
    print(f"GPT: {ai_message.content}")  # GPT 응답 출력

    # 도구 호출 처리
    tool_calls = ai_message.tool_calls  # AI 응답에 포함된 tool_calls 가져오기
    if tool_calls:  # tool_calls가 있는 경우
        for tool_call in tool_calls:
            tool_name = tool_call.function.name  # 호출할 함수 이름
            tool_call_id = tool_call.id  # tool_call ID
            arguments = json.loads(tool_call.function.arguments)  # 매개변수 파싱

            # 호출할 함수 실행
            if tool_name == "get_current_time":
                func_result = get_current_time(timezone=arguments["timezone"])
            elif tool_name == "get_yf_stock_info":
                func_result = get_yf_stock_info(ticker=arguments["ticker"])
            elif tool_name == "get_yf_stock_history":
                func_result = get_yf_stock_history(
                    ticker=arguments["ticker"], period=arguments["period"]
                )
            elif tool_name == "get_yf_stock_recommendations":
                func_result = get_yf_stock_recommendations(ticker=arguments["ticker"])
            else:
                func_result = "알 수 없는 도구 호출입니다."

            # 함수 실행 결과를 대화 기록에 추가
            messages.append(
                {
                    "role": "function",
                    "tool_call_id": tool_call_id,
                    "name": tool_name,
                    "content": func_result,
                }
            )

        # 도구 호출 결과를 바탕으로 GPT 응답 생성
        messages.append(
            {"role": "system", "content": "이제 주어진 결과를 바탕으로 답변할 차례다."}
        )
        ai_response = get_ai_response(messages, tools=tools)
        ai_message = ai_response.choices[0].message

        # 최종 GPT 응답 출력
        print(f"GPT: {ai_message.content}")
        messages.append({"role": "assistant", "content": ai_message.content})

GPT와 대화를 시작합니다. 'exit'을 입력하면 종료됩니다.
GPT: 안녕하세요! 무엇을 도와드릴까요?
GPT: None
2025-11-02 15:53:32 Asia/Seoul
GPT: 현재 서울 시간은 2025년 11월 2일 오후 3시 53분입니다. 
GPT: None
{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '(408) 996-1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple Vision Pro, Apple TV, Apple Watch, Beats products, and HomePod, as well as Apple branded and third-party accessories. It also provides Ap